In [21]:
from pdf_gene import MoonCalc ,generate_pdf
from datetime import datetime
import requests
from tqdm import tqdm
import pandas as pd
from google import genai
import os
from dotenv import load_dotenv
import json
import markdown
import pdfkit
from markdown_pdf import MarkdownPdf, Section

In [23]:
# Path to your .env file (outside the project folder)
load_dotenv(dotenv_path='../.env')

api_key = os.getenv("GEMINI_API_KEY")

In [67]:
def final_df(date,month, year, path="D:/code/Data_2035", dst='D:/Output'):
    
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
#     Moon.pdf()
    return Moon.calculate()
final_df = final_df("27-05-2025","ZUL-HAJJAH","1446")

In [68]:
df = final_df
# Split 'STATION(Sunset)' into 'Station' and 'SunsetTime'
df[['Station', 'SunsetTime']] = df['STATION(Sunset)'].str.extract(r'^(.*?)\s*\((.*?)\)$')

# Optional: drop the original column if no longer needed
df.drop(columns=['STATION(Sunset)'], inplace=True)

# Reorder columns if desired
df = df[['Station', 'SunsetTime', 'LAG TIME(Minutes)', 'MOON ALTITUDE(Degrees)',
         'SUN_AZIMUTH(Degrees)', 'DAZ(Degrees)', 'ELONGATION(Degrees)', 
         'ILLUMINATION(%)', 'CRITERION']]

row = df
row.columns = [
    'Station', 'SunsetTime', 'LagTime', 'MoonAltitude', 'SunAzimuth',
    'DAZ', 'Elongation', 'Illumination', 'Criterion'
]

In [69]:
visibility_mapping = {
    'A': 'Easily visible',
    'B': 'Visible under perfect conditions',
    'C': 'May need optical aid to find the crescent Moon',
    'D': 'Will need optical aid to find the crescent Moon',
    'E': 'Not visible with a telescope',
    'F': 'Not visible, below the Danjon limit'
}

In [134]:
def make_prompt(row):
    # Visibility criterion description
    visibility_description = visibility_mapping.get(row['Criterion'], 'Unknown visibility criterion')

    # Interpret illumination
    illumination = float(row['Illumination'])
    illumination_description = (
        "The illumination is high, indicating good visibility."
        if illumination > 0.8 else
        "The illumination is low, which might make the Moon harder to see."
    )
    # Interpret moon altitude
    moon_altitude = float(row['MoonAltitude'])
    altitude_description = (
        "The Moon is high above the horizon, aiding visibility."
        if moon_altitude > 8 else
        "The Moon is low on the horizon, which may limit visibility."
    )
    return (
        "Here is the required format:\n"
        "{\n"
        "  \"Station\": \"...\",\n"
        "  \"visibility criterion\": \"...\",\n"
        "  \"Time\": \"...\",\n"
        "  \"Date\": \"...\",\n"
        "  \"Conclusion\": \"...\"\n"
        "}\n"
        "Answer based on the following data:\n"
        f"On {row.name}, at station {row['Station']}, "
        f"at the time of Sunset {row["SunsetTime"]},"
        f"moon altitude {moon_altitude}°, "
        f"illumination {illumination}%, ",
        f"and visibility criterion '{row['Criterion']}': {visibility_description}. ",
        f"{illumination_description} {altitude_description} ,"
        f"Based on this data, explain whether the moon is likely to be visible and why.",
    )

def query_gemini(prompt):

    client = genai.Client(api_key=api_key)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    return(response.text)

# ✅ Function to query LLaMA 3.2 via Ollama
def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"].strip()

tqdm.pandas()  # This enables tqdm integration with pandas

df['llama_response'] = df.progress_apply(lambda row: query_gemini(make_prompt(row)), axis=1)
print(df[['Station', 'llama_response']].head())

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:25<00:00,  1.95s/it]

              Station                                     llama_response
date                                                                    
2025-05-27     Cherat  ```json\n{\n  "Station": "Cherat",\n  "visibil...
2025-05-27        Dir  ```json\n{\n  "Station": "Dir",\n  "visibility...
2025-05-27     Gilgit  ```json\n{\n  "Station": "Gilgit",\n  "visibil...
2025-05-27  Islamabad  ```json\n{\n  "Station": "Islamabad",\n  "visi...
2025-05-27     Jiwani  ```json\n{\n  "Station": "Jiwani",\n  "visibil...



C:\Users\Administrator\AppData\Local\Temp\ipykernel_19892\1692458787.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llama_response'] = df.progress_apply(lambda row: query_gemini(make_prompt(row)), axis=1)


In [137]:
import json
import re

def sanitize_and_parse_json(text):
    try:
        # Try parsing directly first
        return json.loads(text)
    except json.JSONDecodeError:
        # If direct parsing fails, try extracting JSON manually
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match:
            try:
                cleaned_text = match.group()
                return json.loads(cleaned_text)
            except json.JSONDecodeError:
                pass
        return None


In [155]:
df["llama_response"]
text = []
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for content in df["llama_response"]:
        response_text = content  # from Gemini
        parsed_data = sanitize_and_parse_json(response_text)

        if parsed_data:
            text.append(parsed_data)
        else:
            print("Invalid JSON after cleaning")


In [156]:
explanations_list = text  # your collected explanations

# visibility_description = visibility_mapping.get(row['Criterion'], 'Unknown visibility criterion')

prompt = f"""

You are provided with specific data about the moon's visibility, including altitude, illumination, and visibility criterion.

Task:
- Write no introduction
- Summarizing the text.
- Write Conclusion that says if moon visibility is high or low.
- Write a structured report of findings.
"""

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt]
)

In [157]:
print(response.text)

## Summary

The provided data assesses moon visibility based on altitude, illumination, and a visibility criterion. The assessment considers the moon's height above the horizon (altitude) and the percentage of the moon that is lit by the sun (illumination). A specific criterion, which is likely a combined threshold or rule based on altitude and illumination, is used to determine if the moon's visibility is considered sufficient.

## Report of Findings

**Data Points:**

*   **Altitude:** [Replace with actual altitude data from provided text] (Units: Degrees or similar)
*   **Illumination:** [Replace with actual illumination data from provided text] (Units: Percentage)
*   **Visibility Criterion:** [Replace with actual data related to visibility criterion from provided text. This could be text like "Altitude > 10 degrees AND Illumination > 50%", or numerical data if there are limits set on the altitude and/or illumination]
*   **Result:** [Replace with the actual results. This could be 

In [115]:
# Create a MarkdownPdf object. Set TOC level to 2 and optimize the output
pdf = MarkdownPdf(toc_level=1, optimize=True)

# Add a section with links and content
text =response.text
pdf.add_section(Section(text))

# Set document metadata like title and author
pdf.meta["title"] = "Moon Visibility Report"
pdf.meta["author"] = "Artificial Intelligence"

# Save the generated PDF
pdf.save("moon_visibility_report.pdf")
